In [2]:
import os
import zipfile
import random
from tqdm import tqdm
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
import cv2
#import keras
print(tf.version.VERSION)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.test.gpu_device_name()

2.13.0


'/device:GPU:0'

binary_classification

In [19]:
TRAINING_DIR ="/kaggle/input/binary-multi-retinopathy/data/binary"

train_dataset = tf.keras.utils.image_dataset_from_directory(TRAINING_DIR, validation_split=0.2, 
                                                            subset="training", seed=42, 
                                                            image_size=(340,510),
                                                            label_mode='binary',
                                                             batch_size=128)
val_dataset = tf.keras.utils.image_dataset_from_directory(TRAINING_DIR, validation_split=0.2, 
                                                          subset="validation", seed=42, 
                                                          image_size=(340,510),
                                                          label_mode='binary',
                                                           batch_size=64)

Found 35126 files belonging to 2 classes.
Using 28101 files for training.
Found 35126 files belonging to 2 classes.
Using 7025 files for validation.


In [20]:
# from tensorflow.keras.applications.densenet import DenseNet169
model = tf.keras.models.Sequential([
     tf.keras.layers.Input(shape=(340,510, 3)),
# #     tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
#     tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
#     tf.keras.layers.experimental.preprocessing.RandomHeight(0.1),
#     tf.keras.layers.experimental.preprocessing.RandomWidth(0.1),
#     tf.keras.layers.experimental.preprocessing.Resizing(512,512),
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.applications.resnet.ResNet50(include_top=False,input_shape=(300,300,3),weights='imagenet'),
    
    
    
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
#     tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')

])

#RMSprop(lr=0.0005)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_54 (Conv2D)          (None, 338, 508, 8)       224       
                                                                 
 max_pooling2d_54 (MaxPooli  (None, 169, 254, 8)       0         
 ng2D)                                                           
                                                                 
 conv2d_55 (Conv2D)          (None, 167, 252, 8)       584       
                                                                 
 max_pooling2d_55 (MaxPooli  (None, 83, 126, 8)        0         
 ng2D)                                                           
                                                                 
 conv2d_56 (Conv2D)          (None, 81, 124, 16)       1168      
                                                                 
 max_pooling2d_56 (MaxPooli  (None, 40, 62, 16)       

In [21]:

old =0
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            global old
            if(logs.get('val_acc') > old):
                model.save('voice_model.h5',overwrite=True)
                old = logs.get('val_acc')
saver =myCallback()
history = model.fit(train_dataset,
                              epochs=20,
                              verbose=1,
                              validation_data=val_dataset)#,callbacks=[callbacks])

Epoch 1/20
220/220 [==============================] - 92s 382ms/step - loss: 1.1458 - acc: 0.7257 - val_loss: 125.6819 - val_acc: 0.7349
Epoch 2/20
220/220 [==============================] - 84s 375ms/step - loss: 5.3901 - acc: 0.7297 - val_loss: 1278.6918 - val_acc: 0.7349
Epoch 3/20
220/220 [==============================] - 85s 376ms/step - loss: 0.7721 - acc: 0.7288 - val_loss: 691.9646 - val_acc: 0.7349
Epoch 4/20
220/220 [==============================] - 85s 378ms/step - loss: 522.7771 - acc: 0.7310 - val_loss: 50720.1211 - val_acc: 0.7349
Epoch 5/20
220/220 [==============================] - 84s 375ms/step - loss: 142.3450 - acc: 0.7232 - val_loss: 17279.4102 - val_acc: 0.7349
Epoch 6/20
220/220 [==============================] - 84s 373ms/step - loss: 69.7424 - acc: 0.6819 - val_loss: 30708.7598 - val_acc: 0.7349
Epoch 7/20
220/220 [==============================] - 84s 373ms/step - loss: 29.2983 - acc: 0.6813 - val_loss: 3420.9822 - val_acc: 0.2651
Epoch 8/20
220/220 [=======